In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [ ]:
# Python Program for Floyd Warshall Algorithm 
# Solves all pair shortest path via Floyd Warshall Algrorithm
def floydWarshall(distGraph):
    costMat = []
    pathMat = []
    N = len(distGraph)
    for fromNode in range(N):
        cRow = []
        pRow = []
        for toNode in range(N):
            cRow.append(distGraph[fromNode][toNode])
            pRow.append(str(fromNode)+str(toNode))
        
        costMat.append(cRow)
        pathMat.append(pRow)
            
    for interNode in range(N):
        for fromNode in range(N):
            for toNode in range(N):
                directDist = costMat[fromNode][toNode]
                interDist = costMat[fromNode][interNode] + costMat[interNode][toNode]
                if interDist < directDist:
                    pathMat[fromNode][toNode] = pathMat[fromNode][interNode][:-1] + pathMat[interNode][toNode]
                    costMat[fromNode][toNode] = interDist
    
    return costMat, pathMat

In [ ]:
#testing efficiency: crap I wrote vs flyod-Warshall vs dijsktra
def costFromSource(costMat, sourceNode):
    def getNextWorkNode(workList, N):
        for nodeNo in range(N):
            if workList[nodeNo]:
                return nodeNo  #return first True element
        return -1  # no work left to be done (all False)    N = len(costMat)  #count of nodes including start/end

    N = len(costMat)
    nodeList = [k for k in range(N) if k != sourceNode] #in path from beginNode to endNode, we don't need to re-visit beginNode
    needsWork = [False]*N  #every time we update cost-to-get-to-a-node, set node# element True. We're done when all false
    nodeCosts = [999]*N    #minimum cost-to-get-to-a-node that we know of for each node. Initially all are infinity
    nodePaths = [[] for k in range(N)] #we save minimum cost path-to-reach-node here. We do need to know the paths as well as costs
    needsWork[sourceNode] = True  #start by branching away from the source/begin node
    nodeCosts[sourceNode] = 0     #used on first step to set initial cost=0

    while True:
        fromNode = getNextWorkNode(needsWork, N)  #get next node that needs work (first True element of needsWork)
        if fromNode == -1:  #no work left, we're done
            break

        #print(fromNode)
        curPath = nodePaths[fromNode]      # path taken (list) to reach current node (excluding beginNode). It will include fromNode
        curPathCost = nodeCosts[fromNode]  # cost/distance from following this path     
        newPathLen = len(curPath) + 1    #might use later if we prefer to keep the longest or shortest path when equal cost...

        nodesToVisit = (node for node in nodeList if node not in curPath)  #i think this is roughly as efficient as putting the if inside the loop? generates an iterator
        for toNode in nodesToVisit:  #branch from our fromNode to each other nodes (toNode's)
            edgeCost = costMat[fromNode][toNode]  #get the given cost to travel directly from fromNode to toNode
            newPathCost = edgeCost + curPathCost  #unnecessary intermediate step for clarity
            toNodeCost = nodeCosts[toNode]
            if newPathCost<toNodeCost or (newPathCost==nodeCosts[toNode] and newPathLen > len(nodePaths[toNode])):
                nodeCosts[toNode] = newPathCost         #new minimum cost to reach toNode
                nodePaths[toNode] = curPath + [toNode]  #update best path to toNode with currentPath. Append toNode
                needsWork[toNode] = True                #indicate we need to propagate/branch from toNode given new lower cost to reach

        needsWork[fromNode] = False   #having finished this iteration of while loop, fromNode o longer requires branching/work until/unless it gets updated again
    
    #return nodeCosts
    return nodeCosts, nodePaths

def calcCostMat(costMat):
    minCostMat = []
    minPathMat = []
    for rowNo in range(len(costMat)):
        minCostRow,minPaths = costFromSource(costMat, rowNo)
        minCostMat.append(minCostRow)
        minPathMat.append(minPaths)
    
    return minCostMat, minPathMat

In [ ]:
import random
import time
random.seed()  #seeds from current time. Probably does this automatically on import but i'm too lazy to confirm from doc

def genTestMat(bunnyCount, minTime, maxTime):
    testMat = []
    for rowNo in range(bunnyCount+2):
        testMat.append([random.randint(minTime,maxTime) for x in range(bunnyCount+2)])
        testMat[rowNo][rowNo] = 0

    return testMat

def formLinkList(linkMat):
    linkList = []
    for rowNo in range(len(linkMat)):
        for colNo in range(len(linkMat[0])):
            if rowNo != colNo:
                link = (rowNo, colNo)  #from rowNo to colNo
                cost = linkMat[rowNo][colNo]
                linkList.append([cost, link])
    
    linkList.sort()  #sort by ascending cost (negative/smallest first)
    return linkList

In [ ]:
tMat = genTestMat(7,-1,18)

t0 = time.time()
cMat, cPath = calcCostMat(tMat)
print(time.time() - t0)

t0 = time.time()
cMat2, cPath2 = floydWarshall(tMat)
print(time.time() - t0)

for row in tMat: print(row)

In [ ]:
for row in cMat: print(row)
print()
for row in cMat2: print(row)

In [ ]:
for row in cPath: print(row)
for row in cPath2: print(row)

In [ ]:
with open('test_output_t.txt', 'w') as f:
    for row in tMat:
        f.write(str(row))
        f.write('\n')
        
with open('test_output_c.txt', 'w') as f:
    for row in cMat:
        f.write(str(row))
        f.write('\n')

with open('test_output_c2.txt', 'w') as f:
    for row in cMat2:
        f.write(str(row))
        f.write('\n')